In [7]:
import cv2
import numpy as np
from inference_sdk import InferenceHTTPClient
import os
import tempfile
from scipy.spatial import distance

In [8]:
# Initialize the Roboflow inference client
CLIENT = InferenceHTTPClient(
    api_url="https://outline.roboflow.com",
    api_key="6db3vtCdSMg4O2FRfZZn"
)

MODEL_ID = "data_tonghop/2"

In [9]:
# Folder paths
input_folder = ".\\Sample_P-Weigh"
done_folder = os.path.join(input_folder, "Done")
output_folder = os.path.join(input_folder, "outputs_pig_id")

# Create folders if they don't exist
os.makedirs(done_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Padding for bounding boxes
padding = 10

In [10]:
# Define specific colors (BGR format)
COLORS = [
    (0, 255, 0),    # Green
    (255, 0, 255),  # Pink
    (0, 255, 255),  # Yellow
    (255, 0, 0),    # Blue
    (0, 0, 255),    # Red
    (0, 165, 255)   # Orange
]

In [11]:
# Function to match new bounding boxes to existing tracks
def match_bboxes(new_bboxes, tracked_pigs, max_distance=100):
    new_tracked_pigs = []
    used_indices = set()

    # Calculate centers of new bounding boxes
    new_centers = [((bbox['x1'] + bbox['x2']) / 2, (bbox['y1'] + bbox['y2']) / 2) for bbox in new_bboxes]

    # Match each tracked pig to the closest new bounding box
    for pig in tracked_pigs:
        if not new_centers:
            continue
        last_center = pig['last_center']
        distances = [distance.euclidean(last_center, center) for center in new_centers]
        min_distance_idx = np.argmin(distances)
        if distances[min_distance_idx] < max_distance and min_distance_idx not in used_indices:
            # Update pig with new bounding box
            pig['bbox'] = new_bboxes[min_distance_idx]
            pig['last_center'] = new_centers[min_distance_idx]
            new_tracked_pigs.append(pig)
            used_indices.add(min_distance_idx)

    # Add new pigs for unmatched bounding boxes
    for idx, bbox in enumerate(new_bboxes):
        if idx not in used_indices:
            new_id = len(new_tracked_pigs) + 1
            color_idx = (new_id - 1) % len(COLORS)
            new_tracked_pigs.append({
                'id': new_id,
                'name': f'Pig {new_id}',
                'color': COLORS[color_idx],
                'bbox': bbox,
                'last_center': new_centers[idx]
            })

    return new_tracked_pigs

In [12]:
# Process each video in the folder
for video_file in os.listdir(input_folder):
    if not video_file.endswith(".mp4"):
        continue

    video_path = os.path.join(input_folder, video_file)
    video_name = os.path.splitext(video_file)[0]

    # Create output subfolder for this video
    video_output_folder = os.path.join(output_folder, video_name)
    os.makedirs(video_output_folder, exist_ok=True)

    # Open the video
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calculate frame interval (every half second or 15 frames)
    frame_interval = min(int(fps * 0.5), 15)  # Half a second or 15 frames, whichever is smaller

    # Output video path
    output_video_path = os.path.join(video_output_folder, f"{video_name}_processed.mp4")

    # Video writer to save output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width // 2, frame_height))

    frame_count = 0
    tracked_pigs = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Extract only RGB side (right half of the frame)
        rgb_frame = frame[:, frame_width // 2:]

        # Process bounding boxes every frame_interval
        if frame_count % frame_interval == 0:
            # Save the RGB frame to a temporary file
            with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp_file:
                temp_image_path = temp_file.name
                cv2.imwrite(temp_image_path, rgb_frame)

            try:
                # Perform inference via Roboflow API
                response = CLIENT.infer(temp_image_path, model_id=MODEL_ID)
                
                new_bboxes = []
                for prediction in response['predictions']:
                    x = int(prediction['x'])
                    y = int(prediction['y'])
                    width = int(prediction['width'])
                    height = int(prediction['height'])

                    # Calculate bounding box coordinates
                    x1 = max(x - width // 2 - padding, 0)
                    y1 = max(y - height // 2 - padding, 0)
                    x2 = min(x + width // 2 + padding, rgb_frame.shape[1] - 1)
                    y2 = min(y + height // 2 + padding, rgb_frame.shape[0] - 1)

                    new_bboxes.append({
                        'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2
                    })

                # Match new bounding boxes to tracked pigs
                tracked_pigs = match_bboxes(new_bboxes, tracked_pigs)

            finally:
                # Clean up the temporary file
                if os.path.exists(temp_image_path):
                    os.remove(temp_image_path)

        # Draw bounding boxes and names on current frame
        for pig in tracked_pigs:
            bbox = pig['bbox']
            color = pig['color']
            name = pig['name']

            # Draw rectangle
            cv2.rectangle(rgb_frame, 
                         (bbox['x1'], bbox['y1']), 
                         (bbox['x2'], bbox['y2']), 
                         color, 
                         2)

            # Draw name above the bounding box
            cv2.putText(rgb_frame, name, 
                       (bbox['x1'], bbox['y1'] - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 
                       0.6, 
                       color, 
                       2)

        # Write the RGB frame to the output video
        out.write(rgb_frame)

        frame_count += 1

    # Release resources for this video
    cap.release()
    out.release()

    # Move processed video to "Done" folder
    os.rename(video_path, os.path.join(done_folder, video_file))

In [13]:
print(f"Processing complete. Outputs saved in {output_folder}")

Processing complete. Outputs saved in .\Sample_P-Weigh\outputs_pig_id
